In [1]:
from datetime import date, timedelta
import os
import pickle
import sys
sys.path.append("..")

In [2]:
from abautomator import experiment

In [3]:
import pandas as pd


In [4]:
# %pip install -U pandas

In [5]:
name = "rel_diff_ci"
df = pickle.load(
    open(os.path.join("..", "tests", f"{name}.p"), "rb" )
)

In [6]:
df.head(2)

,exp_cond,metric,tx_mean,tx_std,tx_count,factor_label,ctrl_mean,ctrl_std,ctrl_count,abs_mean,abs_std,upper_68_ci,lower_68_ci,upper_95_ci,lower_95_ci,mean,std
0,Video01,n_user_sessions,2.097734,2.643838,3530.0,"(n_user_sessions, Video01)",2.085166,2.522356,3546.0,0.012567,0.061436,3.549024,-2.343621,6.495346,-5.289944,0.602701,2.946323
1,Video01,pct_user_sessions,0.798584,0.401115,3530.0,"(pct_user_sessions, Video01)",0.823463,0.381330,3546.0,-0.024879,0.009305,-1.891318,-4.151330,-0.761312,-5.281336,-3.021324,1.130006


In [22]:
import math

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import CDSView, ColumnDataSource, IndexFilter, FactorRange, BooleanFilter
from bokeh.palettes import Colorblind7
from bokeh.transform import factor_cmap

from math import pi

output_notebook()

Loading BokehJS ...

In [23]:
[item for item in df["exp_cond"].unique()]

['Video01', 'Video02', 'Carousel01', 'Carousel02', 'Carousel03', 'Carousel04']

In [86]:
def add_error_bars(plot, source, view):
    upper_eb = _get_error_bar("upper_68_ci", "upper_95_ci", plot, source, view)
    lower_eb = _get_error_bar("lower_95_ci", "lower_68_ci", plot, source, view)
    
    return lower_eb, upper_eb
    
    
def _get_error_bar(left_label, right_label, plot, source, view):
    return plot.segment(
        right_label,
        "factor_label",
        left_label,
        "factor_label",
        color="black",
        source=source,
        view=view,
    )

In [87]:
source = ColumnDataSource(df)

view = CDSView(source=source, filters=[IndexFilter([0, 2, 4])])

p = figure(
    y_range=FactorRange(*list(source.data["factor_label"])),
    height=450,
    title=f"exp name",
    toolbar_location=None,
    tools=""
)

for exp_cond in df["exp_cond"].unique():
    bools = [True if exp_cond == filter_cond else False for filter_cond in source.data['exp_cond']]
    view = CDSView(source=source, filters=[BooleanFilter(bools)])
    
    lower_eb, upper_eb = add_error_bars(p, source, view)

    horbar = p.hbar(
        y='factor_label',
        right="upper_68_ci",
        left="lower_68_ci",
        legend_label=exp_cond,
    #     fill_color=factor_cmap(
    #         'exp_cond', palette=Colorblind7, factors=list(set(source.data["exp_cond"])), start=1, end=2
    #     ),
        line_color=None,
        height=0.6,
        fill_alpha=0.8,
        source=source,
        view=view,
    )
    horbar.js_link('visible', lower_eb, 'visible')
    horbar.js_link('visible', upper_eb, 'visible')

p.legend.location = "top_right"
p.legend.click_policy="hide"

p.ygrid.grid_line_color = None
p.xaxis.axis_label = 'Absolute difference from control'

p.yaxis.major_label_text_alpha = 0.0
p.yaxis.major_tick_in = 0
p.yaxis.major_tick_out = 0
p.yaxis.major_label_text_font_size = '1px'

p.yaxis.group_label_orientation = "horizontal"
p.yaxis.separator_line_alpha = 0
p.yaxis.minor_tick_line_color = "yellow"

In [88]:
show(p)

In [56]:
type(hbar)

bokeh.models.renderers.GlyphRenderer